<center>
# N(utrient)-P(hytoplankton)-Z(ooplankton)-D(etritus) Model
<center>
### A toy interactive model of ocean ecosystem dynamics

<center>
Modified from R.X. Brady's simple NPZD model (https://github.com/bradyrx/NPZD-Model)

---
<center>
**References:**

1. J.L. Sarmiento and N. Gruber (2006). Ocean Biogeochemical Dynamics. Chapter 4: "Production."

1. A.M. Edwards (2001). Adding Detritus to a Nutrient–Phytoplankton–Zooplankton Model :A Dynamical-Systems Approach. J Plankton Res.


---
## Summary
We can create a reduced model of a complex lower-trophic ocean ecosystem with a few differential equations. Here, we choose to model four variables: an arbitrary nutrient concentration (generally thought of as a macronutrient such as nitrate), a phytoplankton concentration (maybe a diatom), a zooplankton concentration (with option to change the type of zooplankter), and a detritus concentration (waste products).

In reality, we are just modeling a finite reservoir of nitrate, and considering how it gets redistributed around the ecosystem, given a few initial conditions and parameter settings. In other words, we aren't explicitly modeling phytoplankton cell count or biomass, but rather tracking where the nitrate goes as it is incorporated into organic matter via photosynthesis, or consumed by zooplankton.

Differential equations (DE's) are complex things to deal with. In a model like this, we have four DE's interacting with one another, because the rate of change of the given population (nutrient, phytoplankton, zooplankton, or detritus) is dependent on the current state of the other three populations. Thus, it is a lot easier to discretize a model into time steps, and reduce our DE's into algebraic equations that may be solved in reference to the current state of the system.

Here, I use an explicit time-differencing scheme (forward Euler method) to model this simple ocean ecosystem.

---

## Differential Equations Contributing to the Model
The four raw DE's are as follows (where N is our nutrients, P is our phytoplankton, Z is our zooplankton, and D is our detritus):

$\frac{dN}{dt} = -V_{m}\left(\frac{N}{K_{N}+N}\right)f(I_{0})P~+~\alpha R_{m}\left(1-e^{-\lambda P}\right)Z~+~r P~+~gZ~+~\phi D$

$\frac{dP}{dt} = V_{m}\left(\frac{N}{K_{N} + N}\right)f(I_{0})P~-~R_{m}\left(1 - e^{-\lambda P}\right)Z~-~\epsilon P~-~rP$

$\frac{dZ}{dt} = AE R_{m}\left(1-e^{-\lambda P}\right)Z~-~gZ$

$\frac{dD}{dt} = \epsilon P~+~(1 - \alpha - AE)R_{m}\left(1 - e^{-\lambda P}\right)Z~-~\phi D$

---

### Terms
#### Bulk Terms
If you look closely at each DE, you note that these are simply source (+) minus sink (-) equations. This simple model only has a few nitrogen exchange processes: 

$V_{m}\left(\frac{N}{K_{N} + N}\right)f(I_{0})P$ : Phytoplankton grazing term. How much inorganic nitrogen are they taking up?

$R_{m}\left(1-e^{-\lambda P}\right)Z$ : Zooplankton grazing term. How much nitrogen are they taking up after consuming phytoplankton and releasing some as waste? The $\beta$ coefficient represents the proportion taken up into zooplankton organic matter; the $\alpha$ coefficient is that which is dissolved back into nutrients (perhaps from urine); and (1-$\alpha$-$AE$) is that which is excreted as fecal pellets (to the detritus compartment). 

$\epsilon P$ : How much nitrogen is being returned to the pool from phytoplankton death?

$gZ$ : How much nitrogen is being returned to the pool from zooplankton death?

$rP$ : How much nitrogen is being respired by phytoplankton into detritus?

#### Phytoplankton Terms
$V_{m}$ : Maximum growth rate of an individual plankter (div per day). This value is dependent on temperature, via a lab-derived equation for diatoms: $V_{m} = a\cdot b^{T}$, with $a=0.6d^{-1}$, $b=1.066$, and $c=1(degC)^{-1}$.

$K_{N}$ : Half-saturation constant for nitrogen uptake ($\mu molNl^{-1}$). This is the nitrogen concentration at which the phytoplankton growth rate is at half its maximum value.

$f_{0}$ : Light intensity (0 to 1 weighting function). This is a simple parameterization of a more complex hyperbolic term that uses a similar term to $K_{N}$.

$\epsilon$ : Phytoplankton death rate (cells per day).

$r$ : Respiration rate (per day).

#### Zooplankton Terms
$R_{m}$ : Maximum grazing rate of zooplankton on phytoplankton (cells per day).

$\lambda$ : Grazing constant ($\mu molNl^{-1}$).

$AE$ : Proportion of assimilated nitrogen by zooplankton. In other words, when they graze upon a phytoplankter, how efficient are they at taking up the nitrogen? (dimensionless)

$\alpha$ : Proportion of nitrogen taken up by zooplankton that returns to the environment as dissolved nutrients (urine?).

$g$ : Zooplankton death rate (critters per day).

#### Detritus Terms
$\phi$ : The remineralization rate of detritus back into dissolved nutrients (per day).

---
### Fixed Values/Initial Conditions

|                 Parameter                |   Symbol   |   Default Value   |
|:----------------------------------------:|:----------:|:-----------------:|
| Ambient Temperature                      |      T     |      15 degC      |
| Half-saturation constant for $N$ uptake  |   $K_{N}$  |  1$\mu$mol per L  |
| Maximum Grazing Rate                     |   R$_{m}$  |     1 $d^{-1}$    |
| Zooplankton Death Rate                   |      g     |    0.2$d^{-1}$    |
| Zooplankton Grazing Constant             |  $\lambda$ | 0.2$\mu$mol per L |
| Phytoplankton Death Rate                 | $\epsilon$ |    0.1$d^{-1}$    |
| Proportional Light Intensity             |   f$_{0}$  |        0.25       |
| Zooplankton Dissolved Excretion Fraction |  $\alpha$  |        0.3        |
| Zooplankton Assimilation Efficiency      |      AE    |        0.6        |
| Phytoplankton Respiration Rate           |      r     |        0.15       |
| Detritus Remineralization Rate           |   $\phi$   |    0.4 $d^{-1}$   |

---

### Tools
This model was built using Python 3 and visualized using [Bokeh](http://bokeh.pydata.org/en/latest/).


In [1]:
# Outside packages
import numpy as np

# Bokeh packages
from bokeh.io import output_notebook, show, gridplot, output_file, save
from bokeh.layouts import column, widgetbox
from bokeh.models import CustomJS, ColumnDataSource, Slider, FixedTicker
from bokeh.models.widgets import Slider, Dropdown, RadioButtonGroup
from bokeh.plotting import figure
from bokeh.charts import Area, Bar

# Set up colors (from ColorBrewer discrete colors)
n_color = ["#377eb8"]
p_colors = ["#b7dfb6", "#94cf92", "#70bf6e", "#4daf4a"]
z_colors = ["#f9d1d1", "#f4a3a4", "#ee7576", "#e94749", "#e41a1c"]
d_color = ["#984ea3"]
cmap = n_color + p_colors + z_colors + d_color

In [2]:
# Allow Bokeh to be utilized inline with Jupyter.
output_notebook()

Loading BokehJS ...

# Default Model View

---
We first need to compute the model in Python with some basic parameterizations that we know work. This will serve as the default view when the user opens the page.

In [3]:
# Here we set up the default parameters/coefficients. 
DT = 0.1 # Time Step (in days)
NUM_STEPS = 1000 # Number of time steps to be computed and plotted

f = 0.25 # Light intensity

# Q10 temperature function
T = 15
Tref = 25
Q10 = 2.0
Tfunc = Q10 **((T - Tref)/10.)

# Boltzmann-Arrhenius temperature function
boltz = 8.6173324E-5
Ea = 0.63
T0_Kelvin = 273.15
Tfunc = np.exp(Ea*(T - Tref) / (boltz * (T + T0_Kelvin) * (Tref + T0_Kelvin)))

In [4]:
## Phytoplankton parameters
# range of phytoplankton biomass = 0.2 microns - 2 mm
# size in mm
size_P1 = 0.0005
size_P2 = 0.005
size_P3 = 0.05
size_P4 = 0.5

# note: should try out beta parameters from the literature

# Phytoplankton maximum growth rate (per day)
#Vm1 = 6.0 # Maximum growth rate - phytoplankton 1 (sp)
#Vm2 = 8.0 # Maximum growth rate - phytoplankton 2 (diat)
# solve Vm1 = c * size_P2**beta
# with Vm2 = c * size_P3**beta

beta_vm = 0.1249
c_vm = 11.632
Vm1 = c_vm * size_P1 ** beta_vm
Vm2 = c_vm * size_P2 ** beta_vm
Vm3 = c_vm * size_P3 ** beta_vm
Vm4 = c_vm * size_P4 ** beta_vm

# Other parameters - Phytoplankton
#Kn1 = 0.5    # Half-saturation constant for nitrogen uptake, sp (umolN per l)
#Kn2 = 2.0    # Half-saturation constant for nitrogen uptake, diat (umolN per l)
# solve Kn1 = c * size_P2**beta
# with Kn2 = c * size_P3**beta

beta_kn = 0.6021
c_kn = 12.143
Kn1 = c_kn * size_P1 ** beta_kn
Kn2 = c_kn * size_P2 ** beta_kn
Kn3 = c_kn * size_P3 ** beta_kn
Kn4 = c_kn * size_P4 ** beta_kn

# Mortality rates
#epsilon1 = 0.15  # Phyto death rate - sp (per day)
#epsilon2 = 0.1  # Phyto death rate - diat (per day)
# solve epsi1 = c * size_P2 ** beta
# with epsi2 = c * size_P3 ** beta
beta_epsi = -0.1761
c_epsi = 0.0590
epsilon1 = c_epsi * size_P1 ** beta_epsi
epsilon2 = c_epsi * size_P2 ** beta_epsi
epsilon3 = c_epsi * size_P3 ** beta_epsi
epsilon4 = c_epsi * size_P4 ** beta_epsi

In [5]:
# Zooplankton parameters
# range of zooplankton size: 20 microns - 2 cm
# size in mm
sizes_Z = np.round(np.exp(np.linspace(np.log(0.2), np.log(20), 5)), 3)

# Max zooplankton grazing rates
#Rm1 = 1.20    # Maximum grazing rate - z1 (per day)
#Rm2 = 0.75    # Maximum grazing rate - z2 (per day)
# solve Rm1 = c * sizes_Z[1] ** beta
# with Rm2 = c * sizes_Z[3] ** beta
beta_rm = -0.2041 # Hansen et al. 1997 says -0.16 exponent, so not that far off
c_rm = 1.0929
Rm1 = c_rm * sizes_Z[0]**beta_rm
Rm2 = c_rm * sizes_Z[1]**beta_rm
Rm3 = c_rm * sizes_Z[2]**beta_rm
Rm4 = c_rm * sizes_Z[3]**beta_rm
Rm5 = c_rm * sizes_Z[4]**beta_rm

lambda_Z = 0.2  # Grazing constant (umolN per l)

# Zooplankton linear mortality rate - should decrease with size
#g1_1 = 0.20  # Zooplankton1 (z1) linear mortality rate (per day)
#g2_1 = 0.15  # Zooplankton2 (z2) linear mortality rate (per day)
# solve g1_1 = c * sizes_Z[1] ** beta
# with g2_1 = c * sizes_Z[3] ** beta
beta_g1 = -0.1249
c_g1 = 0.18871
g1_1 = c_g1 * sizes_Z[0]**beta_g1
g1_2 = c_g1 * sizes_Z[1]**beta_g1
g1_3 = c_g1 * sizes_Z[2]**beta_g1
g1_4 = c_g1 * sizes_Z[3]**beta_g1
g1_5 = c_g1 * sizes_Z[4]**beta_g1

# Zooplankton5 quadratic mortality rate (per day) - for a closure term
g2_5 = 0.05

# prey selectivity
direct_pref = 0.7
indirect_pref = 0.3
zoo_pref = 0.5

# Detritus-related stuff.
alpha = 0.3 # Fraction of zoo. uptake that goes immediately to dissolved nutrients.
ae_p = 0.6  # Assimilation efficiency of zooplankton grazing on phytoplankton.
ae_z = 0.7  # Assimilation efficiency of zooplankton predating on zooplankton.
r = 0.15 # Respiration rate.
phi = 0.4 # Remineralization rate of detritus.

In [6]:
# Set Initial Conditions (umol per L)
N_0 = 4 
P1_0 = 3.5
P2_0 = 3.0
P3_0 = 2.5
P4_0 = 2.0 
Z1_0 = 2.5
Z2_0 = 2.0
Z3_0 = 1.5
Z4_0 = 1.0
Z5_0 = 0.5
D_0 = 0

In [7]:
# Initialize Arrays
N = np.empty(NUM_STEPS, dtype="float")
P1 = np.empty(NUM_STEPS, dtype="float")
P2 = np.empty(NUM_STEPS, dtype="float")
P3 = np.empty(NUM_STEPS, dtype="float")
P4 = np.empty(NUM_STEPS, dtype="float")
Z1 = np.empty(NUM_STEPS, dtype="float")
Z2 = np.empty(NUM_STEPS, dtype="float")
Z3 = np.empty(NUM_STEPS, dtype="float")
Z4 = np.empty(NUM_STEPS, dtype="float")
Z5 = np.empty(NUM_STEPS, dtype="float")
D = np.empty(NUM_STEPS, dtype="float")

# Insert Initial Values
N[0]  = N_0
P1[0] = P1_0
P2[0] = P2_0
P3[0] = P3_0
P4[0] = P4_0
Z1[0] = Z1_0
Z2[0] = Z2_0
Z3[0] = Z3_0
Z4[0] = Z4_0
Z5[0] = Z5_0
D[0]  = D_0

# Compute Simulation in Python

In [8]:
# Here we use the Euler forward method to solve for t+1 and reference t. 
for idx in np.arange(1, NUM_STEPS, 1):
    t = idx - 1
    
    # Common terms for simpler code
    gamma_N1   = N[t] / (Kn1 + N[t])
    gamma_N2   = N[t] / (Kn2 + N[t])
    gamma_N3   = N[t] / (Kn3 + N[t])
    gamma_N4   = N[t] / (Kn4 + N[t])
    
    nut_uptake1 = Vm1*Tfunc*gamma_N1*f*P1[t]
    nut_uptake2 = Vm2*Tfunc*gamma_N2*f*P2[t]
    nut_uptake3 = Vm3*Tfunc*gamma_N3*f*P3[t]
    nut_uptake4 = Vm4*Tfunc*gamma_N4*f*P4[t]
    
    zoo1_graze_phyto1 = Rm1 * (1 - np.exp(-lambda_Z * direct_pref * P1[t])) * Z1[t] * Tfunc
    zoo2_graze_phyto1 = Rm2 * (1 - np.exp(-lambda_Z * indirect_pref * P1[t])) * Z2[t] * Tfunc
    zoo2_graze_phyto2 = Rm2 * (1 - np.exp(-lambda_Z * direct_pref * P2[t])) * Z2[t] * Tfunc
    zoo3_graze_phyto2 = Rm2 * (1 - np.exp(-lambda_Z * indirect_pref * P2[t])) * Z3[t] * Tfunc
    zoo3_graze_phyto3 = Rm3 * (1 - np.exp(-lambda_Z * direct_pref * P3[t])) * Z3[t] * Tfunc
    zoo4_graze_phyto3 = Rm4 * (1 - np.exp(-lambda_Z * indirect_pref * P3[t])) * Z4[t] * Tfunc
    zoo4_graze_phyto4 = Rm4 * (1 - np.exp(-lambda_Z * P4[t])) * Z4[t] * Tfunc

    zoo1_graze_det = Rm1 * (1 - np.exp(-lambda_Z * indirect_pref * D[t])) * Z1[t] * Tfunc
    zoo2_graze_det = Rm1 * (1 - np.exp(-lambda_Z * indirect_pref * D[t])) * Z2[t] * Tfunc

    zoo2_prey_zoo1 = Rm2 * (1 - np.exp(-lambda_Z * zoo_pref * Z1[t])) * Z2[t] * Tfunc
    zoo3_prey_zoo2 = Rm3 * (1 - np.exp(-lambda_Z * zoo_pref * Z2[t])) * Z3[t] * Tfunc
    zoo4_prey_zoo3 = Rm4 * (1 - np.exp(-lambda_Z * zoo_pref * Z3[t])) * Z4[t] * Tfunc
    zoo5_prey_zoo4 = Rm5 * (1 - np.exp(-lambda_Z * zoo_pref * Z4[t])) * Z5[t] * Tfunc

    
    # Equation calculations
    N[idx] = DT * (-(nut_uptake1 + nut_uptake2 + nut_uptake3 + nut_uptake4) + 
                   alpha*(zoo1_graze_phyto1 + zoo2_graze_phyto1 + zoo2_graze_phyto2 + zoo3_graze_phyto2 + zoo3_graze_phyto3 + zoo4_graze_phyto3 + zoo4_graze_phyto4) + 
                   r*P1[t] + r*P2[t] + r*P3[t] + r*P4[t] + phi*D[t]) + N[t] 
    
    P1[idx] = DT * (nut_uptake1 - zoo1_graze_phyto1 - zoo2_graze_phyto1 - epsilon1*P1[t] - r*P1[t]) + P1[t]
    P2[idx] = DT * (nut_uptake2 - zoo2_graze_phyto2 - zoo3_graze_phyto2 - epsilon2*P2[t] - r*P2[t]) + P2[t]
    P3[idx] = DT * (nut_uptake3 - zoo3_graze_phyto3 - zoo4_graze_phyto3 - epsilon3*P3[t] - r*P3[t]) + P3[t]
    P4[idx] = DT * (nut_uptake4 - zoo4_graze_phyto4 - epsilon4*P4[t] - r*P4[t]) + P4[t]

    Z1[idx] = DT * (ae_p*zoo1_graze_phyto1 + ae_z*zoo1_graze_det - 
                    zoo2_prey_zoo1 - g1_1*Z1[t]) + Z1[t]  
    Z2[idx] = DT * (ae_p*(zoo2_graze_phyto1 + zoo2_graze_phyto2) + ae_z*(zoo2_prey_zoo1 + zoo2_graze_det) - 
                    zoo3_prey_zoo2 - g1_2*Z2[t]) + Z2[t]  
    Z3[idx] = DT * (ae_p*(zoo3_graze_phyto2 + zoo3_graze_phyto3) + ae_z*zoo3_prey_zoo2 - 
                    zoo4_prey_zoo3 - g1_3*Z3[t]) + Z3[t]  
    Z4[idx] = DT * (ae_p*(zoo4_graze_phyto3 + zoo4_graze_phyto4) + ae_z*zoo4_prey_zoo3 - 
                    zoo5_prey_zoo4 - g1_4*Z4[t]) + Z4[t] 
    Z5[idx] = DT * (ae_z*zoo5_prey_zoo4 - g1_5*Z5[t] - g2_5*Z5[t]*Z5[t]) + Z5[t] 
    
    D[idx] = DT * (epsilon1*P1[t] + epsilon2*P2[t] + epsilon3*P3[t] + epsilon4*P4[t] + 
                   g1_1*Z1[t] + g1_2*Z2[t] + g1_3*Z3[t] + g1_4*Z4[t] + g1_5*Z5[t] + g2_5*Z5[t]*Z5[t] + 
                   (1-alpha-ae_p)*(zoo1_graze_phyto1 + zoo2_graze_phyto1 + zoo2_graze_phyto2 + zoo3_graze_phyto2 + zoo3_graze_phyto3 + zoo4_graze_phyto3 + zoo4_graze_phyto4) +  
                   (1-ae_z)*(zoo2_prey_zoo1 + zoo3_prey_zoo2 + zoo4_prey_zoo3 + zoo5_prey_zoo4) - ae_z*(zoo1_graze_det + zoo2_graze_det) -
                   phi*D[t]) + D[t]

# Set up Bokeh Data Structure

In [9]:
x = np.arange(1, NUM_STEPS + 1, 1)
N = N
P1 = P1
P2 = P2
P3 = P3
P4 = P4
Z1 = Z1
Z2 = Z2
Z3 = Z3
Z4 = Z4
Z5 = Z5
D = D

# Bokeh likes reading data via its own version of dictionaries.
source = ColumnDataSource(data = {
        'x'    : x,
        'N'    : N,
        'P1'   : P1,
        'P2'   : P2,
        'P3'   : P3,
        'P4'   : P4,        
        'Z1'   : Z1,
        'Z2'   : Z2,
        'Z3'   : Z3,
        'Z4'   : Z4,
        'Z5'   : Z5,
        'D'    : D,
        'P1sum' : N + P1,# These sum variables are used for a stacked bar plot.
        'P2sum' : N + P1 + P2,
        'P3sum' : N + P1 + P2 + P3,
        'P4sum' : N + P1 + P2 + P3 + P4,
        'Z1sum' : N + P1 + P2 + P3 + P4 + Z1,
        'Z2sum' : N + P1 + P2 + P3 + P4 + Z1 + Z2,
        'Z3sum' : N + P1 + P2 + P3 + P4 + Z1 + Z2 + Z3,
        'Z4sum' : N + P1 + P2 + P3 + P4 + Z1 + Z2 + Z3 + Z4,
        'Z5sum' : N + P1 + P2 + P3 + P4 + Z1 + Z2 + Z3 + Z4 + Z5,
        'Dsum'  : N + P1 + P2 + P3 + P4 + Z1 + Z2 + Z3 + Z4 + Z5 + D,
    })

In [10]:
# Functions for plotting
def plotlines(plot, x, y, source, legend, line_width=3, line_alpha=0.75,
             color='black'):
    plot.line(x, y, source=source, line_width=line_width,
             line_alpha=line_alpha, color=color, legend=legend)

# Standard Visualization (before interaction)

In [11]:
# TIME SERIES PLOT
plot = figure(plot_width=900, plot_height=350,
             toolbar_location="right", tools = "save",
             x_range=(1, NUM_STEPS + 1), title="N-P-Z-D Time Series",
             webgl=True)

# Plot data
plotlines(plot, 'x', 'N', source, "Nutrients", color=cmap[0])
plotlines(plot, 'x', 'P1', source, "Phyto1", color=cmap[1])
plotlines(plot, 'x', 'P2', source, "Phyto2", color=cmap[2])
plotlines(plot, 'x', 'P3', source, "Phyto3", color=cmap[3])
plotlines(plot, 'x', 'P4', source, "Phyto4", color=cmap[4])
plotlines(plot, 'x', 'Z1', source, "Zoo1", color=cmap[5])
plotlines(plot, 'x', 'Z2', source, "Zoo2", color=cmap[6])
plotlines(plot, 'x', 'Z3', source, "Zoo3", color=cmap[7])
plotlines(plot, 'x', 'Z4', source, "Zoo4", color=cmap[8])
plotlines(plot, 'x', 'Z5', source, "Zoo5", color=cmap[9])
plotlines(plot, 'x', 'D', source, "Detritus", color=cmap[10])


# Plot aesthetics
# Title
plot.title.align           = "center"
plot.title.text_font_style = "normal"
plot.title.text_font_size  = "14pt"

# Axes
plot.yaxis.axis_label                 = 'Concentration (umolN per L)'
plot.yaxis.axis_label_text_font_style = "normal"
plot.yaxis.axis_label_text_font_size  = "10pt"
plot.xaxis.axis_label                 = 'Model Days'
plot.xaxis.axis_label_text_font_style = "normal"
plot.xaxis.axis_label_text_font_size  = "10pt"

# Grid
plot.ygrid.grid_line_alpha       = 0.2
plot.xgrid.grid_line_alpha       = 0
plot.xgrid.minor_grid_line_color = 'grey'
plot.xgrid.minor_grid_line_alpha = 0.2

In [12]:
# STACKED BAR PLOT
plot2 = figure(plot_width=900, plot_height=350,
            toolbar_location="right", tools = "save",
            x_range=(1, NUM_STEPS + 1), y_range=plot.y_range, 
            title="Nutrient Distribution", webgl=True)

# Plot data
plot2.vbar('x', width=0.2, bottom=0, top='N', source=source, color=cmap[0])
plot2.vbar('x', width=0.2, bottom='N', top='P1sum', source=source, color=cmap[1])
plot2.vbar('x', width=0.2, bottom='P1sum', top='P2sum', source=source, color=cmap[2])
plot2.vbar('x', width=0.2, bottom='P2sum', top='P3sum', source=source, color=cmap[3])
plot2.vbar('x', width=0.2, bottom='P3sum', top='P4sum', source=source, color=cmap[4])
plot2.vbar('x', width=0.2, bottom='P4sum', top='Z1sum', source=source, color=cmap[5])
plot2.vbar('x', width=0.2, bottom='Z1sum', top='Z2sum', source=source, color=cmap[6])
plot2.vbar('x', width=0.2, bottom='Z2sum', top='Z3sum', source=source, color=cmap[7])
plot2.vbar('x', width=0.2, bottom='Z3sum', top='Z4sum', source=source, color=cmap[8])
plot2.vbar('x', width=0.2, bottom='Z4sum', top='Z5sum', source=source, color=cmap[9])
plot2.vbar('x', width=0.2, bottom='Z5sum', top='Dsum', source=source, color=cmap[10])

# Aesthetics
plot2.y_range.start = 0

# Title
plot2.title.align           = "center"
plot2.title.text_font_style = "normal"
plot2.title.text_font_size   = "14pt"

# Axes
plot2.yaxis.axis_label                 = 'Concentration (umolN per L)'
plot2.yaxis.axis_label_text_font_style = "normal"
plot2.yaxis.axis_label_text_font_size  = "10pt"
plot2.xaxis.axis_label                 = 'Model Days'
plot2.xaxis.axis_label_text_font_style = "normal"
plot2.xaxis.axis_label_text_font_size  = "10pt"

In [13]:
# TIME SERIES PLOT
plot3 = figure(plot_width=900, plot_height=350,
             toolbar_location="right", tools = "save",
             x_range=(1, NUM_STEPS + 1), title="Phytoplankton Types Time Series",
             webgl=True)

# Plot data
plotlines(plot3, 'x', 'P1', source, "Phyto1", color=cmap[1])
plotlines(plot3, 'x', 'P2', source, "Phyto2", color=cmap[2])
plotlines(plot3, 'x', 'P3', source, "Phyto3", color=cmap[3])
plotlines(plot3, 'x', 'P4', source, "Phyto4", color=cmap[4])

# Plot aesthetics
# Title
plot3.title.align           = "center"
plot3.title.text_font_style = "normal"
plot3.title.text_font_size  = "14pt"

# Axes
plot3.yaxis.axis_label                 = 'Concentration (umolN per L)'
plot3.yaxis.axis_label_text_font_style = "normal"
plot3.yaxis.axis_label_text_font_size  = "10pt"
plot3.xaxis.axis_label                 = 'Model Days'
plot3.xaxis.axis_label_text_font_style = "normal"
plot3.xaxis.axis_label_text_font_size  = "10pt"

# Grid
plot3.ygrid.grid_line_alpha       = 0.2
plot3.xgrid.grid_line_alpha       = 0
plot3.xgrid.minor_grid_line_color = 'grey'
plot3.xgrid.minor_grid_line_alpha = 0.2

In [14]:
# TIME SERIES PLOT
plot4 = figure(plot_width=900, plot_height=350,
             toolbar_location="right", tools = "save",
             x_range=(1, NUM_STEPS + 1), title="Zooplankton Types Time Series",
             webgl=True)

# Plot data
plotlines(plot4, 'x', 'Z1', source, "Zoo1", color=cmap[5])
plotlines(plot4, 'x', 'Z2', source, "Zoo2", color=cmap[6])
plotlines(plot4, 'x', 'Z3', source, "Zoo3", color=cmap[7])
plotlines(plot4, 'x', 'Z4', source, "Zoo4", color=cmap[8])
plotlines(plot4, 'x', 'Z5', source, "Zoo5", color=cmap[9])


# Plot aesthetics
# Title
plot4.title.align           = "center"
plot4.title.text_font_style = "normal"
plot4.title.text_font_size  = "14pt"

# Axes
plot4.yaxis.axis_label                 = 'Concentration (umolN per L)'
plot4.yaxis.axis_label_text_font_style = "normal"
plot4.yaxis.axis_label_text_font_size  = "10pt"
plot4.xaxis.axis_label                 = 'Model Days'
plot4.xaxis.axis_label_text_font_style = "normal"
plot4.xaxis.axis_label_text_font_size  = "10pt"

# Grid
plot4.ygrid.grid_line_alpha       = 0.2
plot4.xgrid.grid_line_alpha       = 0 
plot4.xgrid.minor_grid_line_color = 'grey'
plot4.xgrid.minor_grid_line_alpha = 0.2

# Javascript Interaction

In [15]:
callback = CustomJS(args=dict(source=source), code="""
    // Ingest main model data for modification
    var data = source.get('data');
    x     = data['x'];
    N     = data['N'];
    P1    = data['P1'];
    P2    = data['P2'];    
    P3    = data['P3'];
    P4    = data['P4'];
    Z1    = data['Z1'];
    Z2    = data['Z2'];
    Z3    = data['Z3'];
    Z4    = data['Z4'];
    Z5    = data['Z5'];
    D     = data['D'];
    
    // Again, these are optional and are to serve a stacked bar plot visualization.
    P1sum = data['P1sum'];
    P2sum = data['P2sum'];
    P3sum = data['P3sum'];
    P4sum = data['P4sum'];
    Z1sum = data['Z1sum'];
    Z2sum = data['Z2sum'];
    Z3sum = data['Z3sum'];
    Z4sum = data['Z4sum'];
    Z5sum = data['Z5sum'];
    Dsum = data['Dsum'];
    
    // -------------- Parameters ------------------
    var dt = 0.1; // Anything that is fixed and unable to be modified just gets the same value as the default view.
    var f = light.get('value'); // Anything that will be a slider or button gets this JS call.
    var T = temperature.get('value');

    // Boltzmann-Arrhenius temperature function
    var boltz = 8.6173324e-5;
    var Ea = 0.63;
    var Tref = 25;
    var T0_Kelvin = 273.15;
    var Tfunc = Math.exp(Ea*(T - Tref) / (boltz * (T + T0_Kelvin) * (Tref + T0_Kelvin)));

    // -- Phytoplankton parameters --
    var size_P1 = 0.0005;
    var size_P2 = 0.005;
    var size_P3 = 0.05;
    var size_P4 = 0.5;

    // Phytoplankton maximum growth rate (per day)
    var beta_vm = beta_vm.get('value'); // 0.1249;
    var c_vm = 11.632;
    var Vm1 = c_vm * Math.pow(size_P1, beta_vm);
    var Vm2 = c_vm * Math.pow(size_P2, beta_vm);
    var Vm3 = c_vm * Math.pow(size_P3, beta_vm);
    var Vm4 = c_vm * Math.pow(size_P4, beta_vm);

    // Phytoplankton half-saturation constant for nutrient uptake
    var beta_kn = beta_kn.get('value'); // 0.6021;
    var c_kn = 12.143;
    var Kn1 = c_kn * Math.pow(size_P1, beta_kn);
    var Kn2 = c_kn * Math.pow(size_P2, beta_kn);
    var Kn3 = c_kn * Math.pow(size_P3, beta_kn);
    var Kn4 = c_kn * Math.pow(size_P4, beta_kn);

    // Phytoplankton mortality rates
    var beta_epsi = beta_epsi.get('value'); // -0.1761;
    var c_epsi = 0.0590;
    var epsilon1 = c_epsi * Math.pow(size_P1, beta_epsi);
    var epsilon2 = c_epsi * Math.pow(size_P2, beta_epsi);
    var epsilon3 = c_epsi * Math.pow(size_P3, beta_epsi);
    var epsilon4 = c_epsi * Math.pow(size_P4, beta_epsi);
    
    // -- Zooplankton parameters --
    var size_Z1 = 0.2;
    var size_Z2 = 0.632;
    var size_Z3 = 2.0;
    var size_Z4 = 6.325;
    var size_Z5 = 20;

    // Max zooplankton grazing rates
    var beta_rm = beta_rm.get('value'); // -0.2041;
    var c_rm = 1.0929;
    var Rm1 = c_rm * Math.pow(size_Z1, beta_rm);
    var Rm2 = c_rm * Math.pow(size_Z2, beta_rm);
    var Rm3 = c_rm * Math.pow(size_Z3, beta_rm);
    var Rm4 = c_rm * Math.pow(size_Z4, beta_rm);
    var Rm5 = c_rm * Math.pow(size_Z5, beta_rm);

    var lambda_Z = 0.2;  # Grazing constant (umolN per l)

    // Zooplankton linear mortality rate
    var beta_g1 = beta_g1.get('value'); // -0.1249;
    var c_g1 = 0.18871;
    var g1_1 = c_g1 * Math.pow(size_Z1, beta_g1);
    var g1_2 = c_g1 * Math.pow(size_Z2, beta_g1);
    var g1_3 = c_g1 * Math.pow(size_Z3, beta_g1);
    var g1_4 = c_g1 * Math.pow(size_Z4, beta_g1);
    var g1_5 = c_g1 * Math.pow(size_Z5, beta_g1);

    // Zooplankton5 quadratic mortality rate (per day)
    var g2_5 = g2_5.get('value'); // 0.05;

    // Zooplankton prey selectivity
    var direct_pref = 0.7;
    var indirect_pref = 0.3;
    var zoo_pref = 0.5;

    // -- Detritus-Related Parameters --
    var alpha = 0.3;
    var ae_p = 0.6;
    var ae_z = 0.7;
    var r = 0.15;
    var phi = 0.4;    
    

    // Need to use IF statements for the button widget.
    // var entry = zooSpecies.get('active');
    //if (entry === 0) {
    //    var Rm = 1.6;
    //} else if (entry === 1) {
    //    var Rm = 1.8;
    //} else if (entry === 2) {
    //    var Rm = 1;
    //} else {
    //    var Rm = 2;
    //}


    // --- Initial Conditions with modifications allowed ---
    var N_0  = nut.get('value');
    var P1_0 = phyto1.get('value');
    var P2_0 = phyto2.get('value');
    var P3_0 = phyto3.get('value');
    var P4_0 = phyto4.get('value');
    var Z1_0 = zoo1.get('value');
    var Z2_0 = zoo2.get('value');
    var Z3_0 = zoo3.get('value');
    var Z4_0 = zoo4.get('value');
    var Z5_0 = zoo5.get('value');
    var D_0  = det.get('value');
    
    // Insert Initial Values for model
    N[0]    = N_0;
    P1[0]   = P1_0;
    P2[0]   = P2_0;
    P3[0]   = P3_0;
    P4[0]   = P4_0;
    Z1[0]   = Z1_0;
    Z2[0]   = Z2_0;
    Z3[0]   = Z3_0;
    Z4[0]   = Z4_0;
    Z5[0]   = Z5_0;
    D[0]    = D_0;
    P1sum[0] = N_0 + P1_0;
    P2sum[0] = N_0 + P1_0 + P2_0;
    P3sum[0] = N_0 + P1_0 + P2_0 + P3_0;
    P4sum[0] = N_0 + P1_0 + P2_0 + P3_0 + P4_0;
    Z1sum[0] = N_0 + P1_0 + P2_0 + P3_0 + P4_0 + Z1_0;
    Z2sum[0] = N_0 + P1_0 + P2_0 + P3_0 + P4_0 + Z1_0 + Z2_0;
    Z3sum[0] = N_0 + P1_0 + P2_0 + P3_0 + P4_0 + Z1_0 + Z2_0 + Z3_0;
    Z4sum[0] = N_0 + P1_0 + P2_0 + P3_0 + P4_0 + Z1_0 + Z2_0 + Z3_0 + Z4_0;
    Z5sum[0] = N_0 + P1_0 + P2_0 + P3_0 + P4_0 + Z1_0 + Z2_0 + Z3_0 + Z4_0 + Z5_0;
    Dsum[0]  = N_0 + P1_0 + P2_0 + P3_0 + P4_0 + Z1_0 + Z2_0 + Z3_0 + Z4_0 + Z5_0 + D_0;

    // Run Model
    for (i = 1; i < x.length; i++) {
         t = i - 1;
        
        // Common terms for simpler code
        gamma_N1   = N[t] / (Kn1 + N[t]);
        gamma_N2   = N[t] / (Kn2 + N[t]);
        gamma_N3   = N[t] / (Kn3 + N[t]);
        gamma_N4   = N[t] / (Kn4 + N[t]);
        
        nut_uptake1 = Vm1 * Tfunc * gamma_N1 * f * P1[t];
        nut_uptake2 = Vm2 * Tfunc * gamma_N2 * f * P2[t];
        nut_uptake3 = Vm3 * Tfunc * gamma_N3 * f * P3[t];
        nut_uptake4 = Vm4 * Tfunc * gamma_N4 * f * P4[t];
        
        zoo1_graze_phyto1 = Rm1 * (1 - Math.exp(-lambda_Z * direct_pref * P1[t])) * Z1[t] * Tfunc;
        zoo2_graze_phyto1 = Rm2 * (1 - Math.exp(-lambda_Z * indirect_pref * P1[t])) * Z2[t] * Tfunc;
        zoo2_graze_phyto2 = Rm2 * (1 - Math.exp(-lambda_Z * direct_pref * P2[t])) * Z2[t] * Tfunc;
        zoo3_graze_phyto2 = Rm2 * (1 - Math.exp(-lambda_Z * indirect_pref * P2[t])) * Z3[t] * Tfunc;
        zoo3_graze_phyto3 = Rm3 * (1 - Math.exp(-lambda_Z * direct_pref * P3[t])) * Z3[t] * Tfunc;
        zoo4_graze_phyto3 = Rm4 * (1 - Math.exp(-lambda_Z * indirect_pref * P3[t])) * Z4[t] * Tfunc;
        zoo4_graze_phyto4 = Rm4 * (1 - Math.exp(-lambda_Z * P4[t])) * Z4[t] * Tfunc;

        zoo1_graze_det = Rm1 * (1 - Math.exp(-lambda_Z * indirect_pref * D[t])) * Z1[t] * Tfunc;
        zoo2_graze_det = Rm2 * (1 - Math.exp(-lambda_Z * indirect_pref * D[t])) * Z2[t] * Tfunc;

        zoo2_prey_zoo1 = Rm2 * (1 - Math.exp(-lambda_Z * zoo_pref * Z1[t])) * Z2[t] * Tfunc;
        zoo3_prey_zoo2 = Rm3 * (1 - Math.exp(-lambda_Z * zoo_pref * Z2[t])) * Z3[t] * Tfunc;
        zoo4_prey_zoo3 = Rm4 * (1 - Math.exp(-lambda_Z * zoo_pref * Z3[t])) * Z4[t] * Tfunc;
        zoo5_prey_zoo4 = Rm5 * (1 - Math.exp(-lambda_Z * zoo_pref * Z4[t])) * Z5[t] * Tfunc;
        
        # Equation calculations
        N[i] = dt * (-(nut_uptake1 + nut_uptake2 + nut_uptake3 + nut_uptake4) + alpha*(zoo1_graze_phyto1 + zoo2_graze_phyto1 + zoo2_graze_phyto2 + zoo3_graze_phyto2 + zoo3_graze_phyto3 + zoo4_graze_phyto3 + zoo4_graze_phyto4) + epsilon1*P1[t] + epsilon2*P2[t] + epsilon3*P3[t] + epsilon4*P4[t] + phi*D[t]) + N[t];
        
        P1[i] = dt * (nut_uptake1 - zoo1_graze_phyto1 - zoo2_graze_phyto1 - epsilon1*P1[t] - r*P1[t]) + P1[t];
        P2[i] = dt * (nut_uptake2 - zoo2_graze_phyto2 - zoo3_graze_phyto2 - epsilon2*P2[t] - r*P2[t]) + P2[t];
        P3[i] = dt * (nut_uptake3 - zoo3_graze_phyto3 - zoo4_graze_phyto3 - epsilon3*P3[t] - r*P3[t]) + P3[t];
        P4[i] = dt * (nut_uptake4 - zoo4_graze_phyto4 - epsilon4*P4[t] - r*P4[t]) + P4[t];

        Z1[i] = dt * (ae_p*zoo1_graze_phyto1 + ae_z*zoo1_graze_det - zoo2_prey_zoo1 - g1_1*Z1[t]) + Z1[t];
        Z2[i] = dt * (ae_p*(zoo2_graze_phyto1 + zoo2_graze_phyto2) + ae_z*(zoo2_prey_zoo1 + zoo2_graze_det) - zoo3_prey_zoo2 - g1_2*Z2[t]) + Z2[t];
        Z3[i] = dt * (ae_p*(zoo3_graze_phyto2 + zoo3_graze_phyto3) + ae_z*zoo3_prey_zoo2 - zoo4_prey_zoo3 - g1_3*Z3[t]) + Z3[t];
        Z4[i] = dt * (ae_p*(zoo4_graze_phyto3 + zoo4_graze_phyto4) + ae_z*zoo4_prey_zoo3 - zoo5_prey_zoo4 - g1_4*Z4[t]) + Z4[t];
        Z5[i] = dt * (ae_z*zoo5_prey_zoo4 - g1_5*Z5[t] - g2_5*Z5[t]*Z5[t]) + Z5[t];
        
        D[i] = dt * (r*P1[t] + r*P2[t] + r*P3[t] + r*P4[t] + g1_1*Z1[t] + g1_2*Z2[t] + g1_3*Z3[t] + g1_4*Z4[t] + g1_5*Z5[t] + g2_5*Z5[t]*Z5[t] + (1-alpha-ae_p)*(zoo1_graze_phyto1 + zoo2_graze_phyto1 + zoo2_graze_phyto2 + zoo3_graze_phyto2 + zoo3_graze_phyto3 + zoo4_graze_phyto3 + zoo4_graze_phyto4) + (1-ae_z)*(zoo2_prey_zoo1 + zoo3_prey_zoo2 + zoo4_prey_zoo3 + zoo5_prey_zoo4) - ae_z*(zoo1_graze_det + zoo2_graze_det) - phi*D[t]) + D[t];
        
        // Sum Variables
        P1sum[i] = N[i] + P1[i];
        P2sum[i] = N[i] + P1[i] + P2[i];
        P3sum[i] = N[i] + P1[i] + P2[i] + P3[i];
        P4sum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i];
        Z1sum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i] + Z1[i];
        Z2sum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i] + Z1[i] + Z2[i];
        Z3sum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i] + Z1[i] + Z2[i] + Z3[i];
        Z4sum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i] + Z1[i] + Z2[i] + Z3[i] + Z4[i];
        Z5sum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i] + Z1[i] + Z2[i] + Z3[i] + Z4[i] + Z5[i];
        Dsum[i] = N[i] + P1[i] + P2[i] + P3[i] + P4[i] + Z1[i] + Z2[i] + Z3[i] + Z4[i] + Z5[i] + D[i];
        
        }
    source.trigger('change');
""")

# Building Sliders and Buttons

In [16]:
# Nutrient Initial Conditions
nut_slider = Slider(start = 0, end = 10, value = N_0, step = 0.1, title = "Initial Nutrient Concentration", callback=callback)
callback.args["nut"] = nut_slider # The quoted "nut" is what is called in the JS interaction.

# Phytoplankton Initial Conditions
phyto1_slider = Slider(start = 0, end = 10, value = P1_0, step = 0.1, title = "Initial Phyto_1 Concentration", callback=callback)
callback.args["phyto1"] = phyto1_slider

phyto2_slider = Slider(start = 0, end = 10, value = P2_0, step = 0.1, title = "Initial Phyto_2 Concentration", callback=callback)
callback.args["phyto2"] = phyto2_slider

phyto3_slider = Slider(start = 0, end = 10, value = P3_0, step = 0.1, title = "Initial Phyto_3 Concentration", callback=callback)
callback.args["phyto3"] = phyto3_slider

phyto4_slider = Slider(start = 0, end = 10, value = P4_0, step = 0.1, title = "Initial Phyto_4 Concentration", callback=callback)
callback.args["phyto4"] = phyto4_slider

# Zooplankton Initial Conditions
zoo1_slider = Slider(start = 0, end = 10, value = Z1_0, step = 0.1, title = "Initial Zoo_1 Concentration", callback=callback)
callback.args["zoo1"] = zoo1_slider

zoo2_slider = Slider(start = 0, end = 10, value = Z2_0, step = 0.1, title = "Initial Zoo_2 Concentration", callback=callback)
callback.args["zoo2"] = zoo2_slider

zoo3_slider = Slider(start = 0, end = 10, value = Z3_0, step = 0.1, title = "Initial Zoo_3 Concentration", callback=callback)
callback.args["zoo3"] = zoo3_slider

zoo4_slider = Slider(start = 0, end = 10, value = Z4_0, step = 0.1, title = "Initial Zoo_4 Concentration", callback=callback)
callback.args["zoo4"] = zoo4_slider

zoo5_slider = Slider(start = 0, end = 10, value = Z5_0, step = 0.1, title = "Initial Zoo_5 Concentration", callback=callback)
callback.args["zoo5"] = zoo5_slider

# Detritus Initial Conditions
det_slider = Slider(start = 0, end = 10, value = D_0, step = 0.25, title = "Initial Detritus Concentration", callback=callback)
callback.args["det"] = det_slider

# Ambient Temperature
temp_slider = Slider(start = 0, end = 30, value = T, step = 0.5, title = "Water Temperature (degC)", callback=callback)
callback.args["temperature"] = temp_slider


# Phytoplankton maximum growth rate - power scaling and coefficients
beta_vm_slider = Slider(start = -1, end = 1, value = beta_vm, step = 0.01, title = "Phyto max growth rate scaling", callback=callback)
callback.args["beta_vm"] = beta_vm_slider

# Phytoplankton half-saturation constant
beta_kn_slider = Slider(start = -1, end = 1, value = beta_kn, step = 0.01, title = "Phyto half-sat constant scaling", callback=callback)
callback.args["beta_kn"] = beta_kn_slider

# Phytoplankton mortality rates
beta_epsi_slider = Slider(start = -1, end = 1, value = beta_epsi, step = 0.01, title = "Phyto mortality scaling", callback=callback)
callback.args["beta_epsi"] = beta_epsi_slider

# Max zooplankton grazing rates
beta_rm_slider = Slider(start = -1, end = 1, value = beta_rm, step = 0.01, title = "Zoop grazing scaling", callback=callback)
callback.args["beta_rm"] = beta_rm_slider

# Zooplankton linear mortality rate
beta_g1_slider = Slider(start = -1, end = 1, value = beta_g1, step = 0.01, title = "Zoop mortality scaling", callback=callback)
callback.args["beta_g1"] = beta_g1_slider

# Zooplankton5 quadratic mortality rate (per day)
g2_5_slider = Slider(start = 0, end = 0.5, value = g2_5, step = 0.01, title = "Zoop_5 quadratic mortality rate", callback=callback)
callback.args["g2_5"] = g2_5_slider

# Light Intensity
light_slider = Slider(start = 0, end = 1, value = f, step = 0.05, title = "Proportional Light Intensity", callback=callback)
callback.args["light"] = light_slider

# Final Layout and Saving the Plot

In [17]:
layout = gridplot([[nut_slider, phyto1_slider, phyto2_slider, phyto3_slider], 
                   [phyto4_slider, zoo1_slider, zoo2_slider, zoo3_slider],
                   [zoo4_slider, zoo5_slider, det_slider],
                   [beta_vm_slider, beta_kn_slider, beta_epsi_slider, beta_rm_slider],
                   [beta_g1_slider, g2_5_slider, light_slider, temp_slider], 
                   [plot], [plot2], [plot3], [plot4]])

# Option 1: Display in the Notebook
# show(layout)

# Option 2: Save to HTML to run in browser
output_file('index.html')
save(layout)

'/Users/jluo/Dropbox/Research/NCAR/models/NPZD-Model/index.html'